# Stat 201 Project

Introduction:

Higher education institutions are continually striving to improve student retention and academic success. For this purpose, a comprehensive dataset has been collected, amalgamating information from various disjoint databases related to students enrolled in different undergraduate degree programs in an effort to decrease the failure and dropout rate of students. The institution this data set was taken from offers a wide range of programs, encompassing a diverse range of disciplines, including agronomy, design, education, nursing, journalism, management, social service, and technologies. Higher education has the potential to allow significantly different career paths for individuals who pursue a university or college degree but not every student is well supported in these educational institutes. The understanding of factors that influence academic success, can allow for a curated and informed academic program design to support students better.

Research Question

The central question addressed by this project is as follows: How can we predict students' likelihood of dropping out and their likelihood of academic success in various undergraduate degree programs, given the demographic, socioeconomic, and academic path information available at the time of enrollment?

### not sure exactly what our variables of interest are so I am just adding a few placeholder ones ###

Variables of Interest:

Some of the variables of interest in the project include student mean grades and gender diversity at the university.

Category-based: drop-out, succeed, average (probability of each outcome)

the level of confidence in our predictions and the degree of variability in student outcomes within each category.

### which leads to the end of the intro below   ###

Via the mentioned variables and statistical inference, we can provide valuable insights for universities and colleges to enhance their support systems, ultimately improving the overall success and retention of students across a range of undergraduate degree programs.
